# Create animated-tiles


## Setup

### Library import


In [ ]:
import os
import re
import subprocess
import sys
from pathlib import Path

# import boto3
# import botocore
import dotenv
import numpy as np
import rasterio
from dotenv import load_dotenv
from matplotlib.colors import LinearSegmentedColormap
from rasterio.windows import from_bounds

sys.path.append("../src")

import warnings

from rio_tiler.errors import NoOverviewWarning

from data_processing.animated_tiles import AnimatedTiles
from data_processing.raster_processor import RasterProcessor
from data_processing.utils import batch_reproject_rasters, clip_rasters_by_vector, hex_to_rgba

# from helpers.s3 import delete_s3_folder, upload_local_directory_to_s3
from helpers.vector_to_raster import batch_convert_vectors

# Suppress the specific warning
warnings.filterwarnings('ignore', category=NoOverviewWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning, module='numpy.ma.core')

dotenv.load_dotenv()

## Create animated-tiles from `GeoTIFFs`

### Define variables: Done

This section collects the parameters of layers that have already been created, in case they need to be run again. It's divided in subsections according to the different stories

#### Mongolia - DR

In [ ]:
# MONGOLIA: Dzud Hazard -> Number of frost days
output_folder = "../data/processed/Disaster Resilience/Mongolia/APNGs/Iron Dzud/"
input_folder = "../data/raw/Disaster Resilience/Mongolia/GeoTIFFs/Iron Dzud/"

Z_MAX = 10
Z_MIN = 2
V_MAX = 185
V_MIN = 100

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#f1eef6",  # 0 days
        "#c8d1e6",
        "#91b6d7",
        "#579ec8",
        "#2382b4",
        "#045a8d",  # 185 days
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

In [ ]:
# MONGOLIA: Dzud Synoptic Assessment
output_folder = "../data/processed/Disaster Resilience/Mongolia/APNGs/Dzud Synoptic Assessment/"
input_folder = "../data/raw/Disaster Resilience/Mongolia/GeoTIFFs/Dzud Synoptic Assessment/"

Z_MAX = 10
Z_MIN = 2
V_MAX = 255
V_MIN = 0

cm = {
    0: (0, 0, 0, 0),
    1: (197, 78, 82, 255),
    2: (100, 182, 206, 255),
    3: (93, 79, 143, 255),
    4: (69, 107, 167, 255),
}

In [ ]:
# MONGOLIA: White Dzud -> Snow cover duration (10 cm)
output_folder = "../data/processed/Disaster Resilience/Mongolia/APNGs/White Dzud/"
input_folder = "../data/raw/Disaster Resilience/Mongolia/GeoTIFFs/White Dzud/"

Z_MAX = 10
Z_MIN = 2
V_MAX = 185
V_MIN = 0

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#f1eef6",  # 0 days
        "#c8d1e6",
        "#91b6d7",
        "#579ec8",
        "#2382b4",
        "#045a8d",  # 185 days
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

#### Botswana - Water

In [ ]:
# Botswana: Groundwater Storage Anomalies -> Mean annual groundwater storage balance
output_folder = "../data/processed/Water Management/Botswana/APNGs/Groundwater_Storage_Anomalies/"
input_folder = "../data/raw/Water Management/Botswana/Rasters/Groundwater Storage Anomalies/"
vector_file = "../data/raw/Water Management/Botswana/Vectors/Botswana_Boundary.shp"

Z_MAX = 8
Z_MIN = 3
V_MAX = 429
V_MIN = 0

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#ca0020",  # 0
        "#f4a582",  # 107
        "#f7f7f7",  # 214
        "#92c5de",  # 322
        "#0571b0",  # 429
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

In [ ]:
# Botswana: Grace Groundwater Drought Index -> Mean annual GGDI
output_folder = "../data/processed/Water Management/Botswana/APNGs/Grace_Groundwater_Drought_Index/"
input_folder = "../data/raw/Water Management/Botswana/Rasters/Grace Groundwater Drought index/"
vector_file = "../data/raw/Water Management/Botswana/Vectors/Botswana_Boundary.shp"

Z_MAX = 8
Z_MIN = 3
V_MAX = 1
V_MIN = -4

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#b30101",
        "#e24a32",
        "#fc8d58",
        "#fff1d9",
        "#ffffff",
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

#### Cote d'Ivoire - Urban

In [ ]:
# Cote d'Ivoire: Percent Impervious Surfaces 1985-2021
output_folder = (
    "../data/processed/Urban Sustainability/Cote d'Ivoire/APNGs/" "PercentImperviousSurfaces/"
)
input_folder = (
    "../data/raw/Urban Sustainability/Cote d'Ivoire/Rasters/"
    "03_Percent_Impervious_Surfaces_1985-2021/COGs/"
)

Z_MAX = 12
Z_MIN = 8
V_MAX = 100
V_MIN = 0

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#EDF8FB",
        "#B3CDE3",
        "#8C96C6",
        "#8856A7",
        "#810F7C",
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}
cm[0] = (0, 0, 0, 0)  # Make 0 transparent

#### Maldives - Marine

In [ ]:
# Maldives: Bathymetry
output_folder = "../data/processed/Marine/Maldives/APNGs/Bathymetry/"
input_folder = "../data/raw/Marine/Maldives/Rasters/"

Z_MAX = 16
Z_MIN = 12
V_MAX = -0.1
V_MIN = -12.5

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#0b0405",
        "#382a54",
        "#36699f",
        "#4dc4ad",
        "#def5e5",
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

In [ ]:
vector_files = [
    "../data/raw/Marine/Maldives/Vectors/UC08-SFC_GulhiMaafushi_2003.shp",
    "../data/raw/Marine/Maldives/Vectors/UC08-SFC_GulhiMaafushi_2022.shp"
]

batch_convert_vectors(
    vector_files=vector_files,
    output_folder="../data/raw/Marine/Maldives/Rasters/Seafloor/",
    field_name="Code",
    reference_raster="../data/raw/Marine/Maldives/Rasters/Bathymetry/UC08-SDB_GulhiMaafushi_2022.tif"
)

In [ ]:
# Maldives: Seafloor Classification - Convert shapefiles to rasters

output_folder = "../data/processed/Marine/Maldives/APNGs/Seafloor/"
input_folder = "../data/raw/Marine/Maldives/Rasters/Seafloor/"

Z_MAX = 16
Z_MIN = 12
V_MAX = 4
V_MIN = 1

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#cc1117",
        "#bbd645",
        "#b133d8",
        "#54da87",
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

#### Tunisia - Marine

In [ ]:
input_folder = Path("../data/raw/Marine/Tunisia/Rasters/phytoplankton/Biomass_carbon_as_net_primary_production_yearly/")
output_folder = Path("../data/raw/Marine/Tunisia/Rasters/phytoplankton/masked/")

raster_files = sorted(input_folder.glob("UC01_PP_yearly_carbon_tons_*.tif"))

masked_rasters = {}


# Loop over rasters and mask 0-values
for raster_path in raster_files:
    year = raster_path.stem.split("_")[-1]  # extract year from filename
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        profile = src.profile

    # Mask 0-values
    data_masked = np.where(data == 0, np.nan, data)  # or use nodata value

    # Store in dictionary by year
    masked_rasters[year] = {"data": data_masked, "profile": profile}

    # Save masked raster
    out_path = output_folder / f"{raster_path.stem}.tif"
    profile.update(dtype=data_masked.dtype, nodata=np.nan)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(data_masked, 1)

print("Masked rasters ready:", list(masked_rasters.keys()))


In [ ]:
# Tunisia: Phytoplankton

output_folder = "../data/processed/Marine/Tunisia/APNGs/Phytoplankton/"
input_folder = "../data/raw/Marine/Tunisia/Rasters/phytoplankton/masked/"

Z_MAX = 9
Z_MIN = 6
V_MAX = 0.007
V_MIN = 0

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#f7fcf5",
        "#c7e9c0",
        "#74c476",
        "#238b45",
        "#00441b"
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

In [ ]:
# Tunisia: Phytoplankton

output_folder = "../data/processed/Marine/Tunisia/APNGs/Seagrass/"
input_folder = "../data/raw/Marine/Tunisia/Rasters/seagrass/"

Z_MAX = 9
Z_MIN = 6
V_MAX = 4
V_MIN = 1

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#009600",
        "#006600",
        "#f9dc86",
        "#0000ff"
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

#### Caribbean - Marine

In [ ]:
# Caribbean Island States

output_folder = "../data/processed/Marine/Caribbean/APNGs/"
input_folder = (
    "../data/raw/Marine/GDA-Marine_ImpactSphere/UC03-WB-Caribbean/"
    "Chlorophyll_from_Sentinel3/"
)

Z_MAX = 10
Z_MIN = 7
V_MAX = 10
V_MIN = 0

# Define the bin edges (intervals) and colors (hex)
intervals = [0, 0.03, 0.5, 1.5, 5, 10]  # these are your class breaks
colors = ["#000000", "#480bfd", "#26ff13", "#f6fa08", "#fc5a09", "#ba0340"]

def hex_to_rgba(hex_color):
    """Convert a hex color string to RGBA tuple.

    Args:
        hex_color (str): Hex color string (e.g., '#FF0000' or 'FF0000')

    Returns:
        tuple: RGBA tuple with values 0-255 and alpha=255
    """
    h = hex_color.lstrip("#")
    return tuple(int(h[i:i+2], 16) for i in (0, 2, 4)) + (255,)

colors_rgba = np.array([hex_to_rgba(c) for c in colors], dtype=np.uint8)

# Normalize the bins to 0-255 scale
intervals_norm = np.clip((np.array(intervals) - V_MIN) / (V_MAX - V_MIN) * 255, 0, 255)

# Map every 0-255 value to its corresponding bin
cmap_uint8 = np.zeros((256, 4), dtype=np.uint8)
bins = intervals_norm
for i in range(256):
    # np.digitize returns the index of the bin the value belongs to
    idx = np.digitize(i, bins, right=False) - 1
    idx = max(0, min(idx, len(colors_rgba)-1))
    cmap_uint8[i] = colors_rgba[idx]

# Final colormap dictionary for rio-tiler
cm = {i: tuple(cmap_uint8[i]) for i in range(256)}

animated_tiles = AnimatedTiles(
    data=input_folder,
    output_folder=output_folder,
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

#### Sarajevo - FFF

In [ ]:
# Sarajevo: CAMS NO2 1km - Reproject rasters to WGS84
# Usage examples:
batch_reproject_rasters(
    input_folder="../data/raw/FFF/Sarajevo/Rasters/CAMS_NO2_1km/",
    output_folder="../data/raw/FFF/Sarajevo/Rasters/CAMS_NO2_1km_reprojected/",
    target_crs="EPSG:4326"  # WGS84
)


In [ ]:
# Mask Sarajevo: CAMS NO2 1km - Mask 0-values in rasters
input_folder = Path("../data/raw/FFF/Sarajevo/Rasters/CAMS_NO2_1km_reprojected/")
output_folder = Path("../data/raw/FFF/Sarajevo/Rasters/CAMS_NO2_1km_masked/")

# Create output folder if it doesn't exist
output_folder.mkdir(parents=True, exist_ok=True)

raster_files = sorted(input_folder.glob("CAMS_NO21KM_*_yearly_mean.tif"))

masked_rasters = {}

# Loop over rasters and mask 0-values
for raster_path in raster_files:
    # Extract year using regex to find 4-digit number
    match = re.search(r'(\d{4})', raster_path.stem)
    year = match.group(1) if match else "unknown"

    with rasterio.open(raster_path) as src:
        data = src.read(1)
        profile = src.profile

    # Mask 0-values
    data_masked = np.where(data == 0, np.nan, data)  # or use nodata value

    # Store in dictionary by year
    masked_rasters[year] = {"data": data_masked, "profile": profile}

    # Save masked raster
    out_path = output_folder / f"{raster_path.stem}.tif"
    profile.update(dtype=data_masked.dtype, nodata=np.nan)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(data_masked, 1)

print("Masked rasters ready:", list(masked_rasters.keys()))

In [ ]:
# Sarajevo: CAMS NO2 1km
output_folder = "../data/processed/FFF/Sarajevo/APNGs/CAMS_NO2_1km/"
input_folder = "../data/raw/FFF/Sarajevo/Rasters/CAMS_NO2_1km_masked/"

Z_MAX = 13
Z_MIN = 8
V_MAX = 31
V_MIN = 1

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#2b83ba",
        "#abdda4",
        "#ffffbf",
        "#fdae61",
        "#d7191c"

    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

#### Mongolia - DR

In [ ]:
# Mongolia: Pasture Biomass - Mask 255 no-data values in rasters
input_folder = Path("../data/raw/Disaster_resilience/Mongolia/Rasters/")
output_folder = Path("../data/raw/Disaster_resilience/Mongolia/Rasters_masked/")

# Create output folder if it doesn't exist
output_folder.mkdir(parents=True, exist_ok=True)

# Look for pasture biomass files with the specific pattern
raster_files = sorted(input_folder.glob("Pasture_Biomass_4C_*.tif"))

if not raster_files:
    print(f"No Pasture_Biomass_4C_*.tif files found in {input_folder}")
    print("Available files:")
    for file in input_folder.glob("*.tif"):
        print(f"  {file.name}")
else:
    print(f"Found {len(raster_files)} pasture biomass files to process")

masked_rasters = {}

# Loop over rasters and mask 255 no-data values
for raster_path in raster_files:
    # Extract date from filename (YYYYMM format)
    # Pattern: Pasture_Biomass_4C_E113-50_N050_202304
    match = re.search(r'(\d{6})$', raster_path.stem)  # 6 digits at end (YYYYMM)
    date = match.group(1) if match else "unknown"
    print(f"Processing {raster_path.name} (date: {date})")

    with rasterio.open(raster_path) as src:
        data = src.read(1)
        profile = src.profile.copy()

        print(f"  Original data range: {data.min()} to {data.max()}")
        print(f"  Original data type: {data.dtype}")

    # Keep original data type and set nodata value to 255
    profile.update(nodata=255)

    # Store original data (don't modify the values, just set nodata in profile)
    masked_rasters[date] = {"data": data, "profile": profile}

    # Save masked raster with simplified filename: Pasture_Biomass_YYYYMM.tif
    out_path = output_folder / f"Pasture_Biomass_{date}01.tif"

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(data, 1)

    # Verify the output
    with rasterio.open(out_path) as check:
        check_data = check.read(1)
        valid_data = check_data[check_data != 255]
        if len(valid_data) > 0:
            print(f"  Output data range: {valid_data.min()} to {valid_data.max()}")
        print(f"  Output nodata value: {check.nodata}")

    print(f"Saved: {out_path.name}")

print(f"Masked rasters ready: {list(masked_rasters.keys())}")

In [ ]:
# Mongolia: pastures
input_folder = "../data/raw/Disaster_resilience/Mongolia/Rasters_masked/"
output_folder = "../data/processed/Disaster_resilience/Mongolia/APNGs/PastureBiomass/"


Z_MAX = 11
Z_MIN = 7
V_MAX = 4
V_MIN = 0

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#dc1010",
        "#e8932c",
        "#1feda8",
        "#50d125",
        "#2d7e2a"

    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

#### Georgia - Water

In [ ]:
# Georgia
output_folder = "../data/processed/Water/Georgia/APNGs/SioniReservoir/"
input_folder = "../data/raw/Water/Georgia/Rasters/Sioni Reservoir sediments/"

Z_MAX = 14
Z_MIN = 12
V_MAX = 200
V_MIN = 0

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#28a8b4",
        "#53aa90",
        "#7eac6c",
        "#aaad48",
        "#d5af25",
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

In [ ]:
# Georgia
output_folder = "../data/processed/Water/Georgia/APNGs/SioniReservoir/"
input_folder = "../data/raw/Water/Georgia/Rasters/Sioni Reservoir sediments/"

Z_MAX = 14
Z_MIN = 12
V_MAX = 200
V_MIN = 0
V_CLIP = 20  # Maximum value for palette

# Create a palette
palette = LinearSegmentedColormap.from_list(
    "palette",
    ["#28a8b4", "#53aa90", "#7eac6c", "#aaad48", "#d5af25"],
    256,
)

# Get 256 RGB colors
x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")

# Map colors to values, clamping at V_CLIP
cm = {}
for i, val in enumerate(np.linspace(0, V_CLIP, 256)):
    cm[int(val)] = tuple(cmap_uint8[i])

# Ensure all values above V_CLIP use the last color
cm.update({v: tuple(cmap_uint8[-1]) for v in range(int(V_CLIP)+1, V_MAX+1)})

animated_tiles = AnimatedTiles(
    data=input_folder,
    output_folder=output_folder,
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

#### Timor-Leste - Water

In [ ]:
# Timor-Leste

input_folder = Path("../data/raw/Water/Timor-Leste/Rasters/Section_2-Water_Seasonality")
output_folder = Path("../data/raw/Water/Timor-Leste/Rasters/Section_2-Water_Seasonality_masked/")
output_folder.mkdir(parents=True, exist_ok=True)

raster_files = sorted(input_folder.glob("Timor-Leste_*_water_seasonality.tif"))

masked_rasters = {}
NEW_NODATA = -9999  # safe nodata value for floats

for raster_path in raster_files:
    match = re.search(r'(\d{4})', raster_path.stem)
    year = match.group(1) if match else "unknown"

    with rasterio.open(raster_path) as src:
        data = src.read(1).astype("float32")  # convert to float
        profile = src.profile.copy()
        original_nodata = src.nodata

    # Mask input nodata AND zeros
    mask = (data == 0)
    if original_nodata is not None:
        mask |= (data == original_nodata)

    data[mask] = np.nan

    masked_rasters[year] = {"data": data, "profile": profile}

    out_path = output_folder / raster_path.name

    # Update profile: float32 + numeric nodata
    profile.update(dtype="float32", nodata=NEW_NODATA)

    # Replace nan with numeric nodata before writing
    data_to_write = np.where(np.isnan(data), NEW_NODATA, data)

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(data_to_write, 1)

print("Masked rasters saved:", list(masked_rasters.keys()))

In [ ]:
# Timor-Leste
output_folder = "../data/processed/Water/Timor-Leste/APNGs/WaterSeasonality2/"
input_folder = "../data/raw/Water/Timor-Leste/Rasters/Section_2-Water_Seasonality_masked/"

Z_MAX = 12
Z_MIN = 12
V_MAX = 12
V_MIN = 1

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#38acff",
        "#253494"
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}


In [ ]:
input_folder = Path("../data/raw/Water/Timor-Leste/Rasters/Section_3-Minimun_Maximum_Extent/Min")
output_folder = Path(
    "../data/raw/Water/Timor-Leste/Rasters/Section_3-Minimun_Maximum_Extent/Min_masked/"
)
output_folder.mkdir(parents=True, exist_ok=True)

raster_files = sorted(input_folder.glob("Timor-Leste_*_minimum_extent.tif"))

# 5 km x 5 km bounding box around (-8.70976, 125.52994)
bbox = (125.50734, -8.73236, 125.55254, -8.68716)


NEW_NODATA = -9999  # safe numeric nodata

masked_rasters = {}

for raster_path in raster_files:
    match = re.search(r'(\d{4})', raster_path.stem)
    year = match.group(1) if match else "unknown"
    with rasterio.open(raster_path) as src:
        # Compute the window that covers the bbox
        window = from_bounds(*bbox, transform=src.transform)
        data = src.read(1, window=window).astype("float32")
        transform = src.window_transform(window)
        profile = src.profile.copy()
        original_nodata = src.nodata

    # Mask zero and original NODATA
    mask = (data == 0)
    if original_nodata is not None:
        mask |= (data == original_nodata)
    data[mask] = np.nan

    # Store clipped masked raster in memory
    masked_rasters[year] = {"data": data, "transform": transform}

    # Prepare output raster
    out_path = output_folder / raster_path.name
    profile.update(
        dtype="float32",
        nodata=NEW_NODATA,
        height=data.shape[0],
        width=data.shape[1],
        transform=transform
    )

    # Convert NaN to nodata before writing
    to_write = np.where(np.isnan(data), NEW_NODATA, data)

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(to_write, 1)

print("Clipped + masked rasters saved:", list(masked_rasters.keys()))

In [ ]:
# Timor-Leste
output_folder = "../data/processed/Water/Timor-Leste/APNGs/WaterMaxExtent/"
input_folder = "../data/raw/Water/Timor-Leste/Rasters/Section_3-Minimun_Maximum_Extent/Max_masked/"

Z_MAX = 15
Z_MIN = 15
V_MAX = 1
V_MIN = 1

# Single blue color for all values
single_color = (56, 172, 255, 255)  # Blue color (R, G, B, Alpha)
cm = dict.fromkeys(range(256), single_color)

#### West Africa - Water

In [ ]:
# West Africa

input_folder = Path("../data/raw/Water/West Africa/Rasters/2_GDAWater_Groundwater_Resources_Estimation_Seasonal/")
vector_file = Path("../data/raw/Water/West Africa/boundaries/west_africa_countries.shp")
clipped_folder = Path("../data/raw/Water/West Africa/Rasters/Clipped_animated/")
output_folder = "../data/processed/Water/West Africa/APNGs/GroundwaterAnomalies/"

clip_rasters_by_vector(input_folder, vector_file, clipped_folder)

Z_MAX = 7
Z_MIN = 2
V_MAX = 144
V_MIN = -144

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#67001F",
        "#E37D63",
        "#F7F6F5",
        "#6CADD1",
        "#003061"
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

animated_tiles = AnimatedTiles(
    data=str(clipped_folder),
    output_folder=str(output_folder),
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

#### Peru & Cameroon - Water

In [ ]:
# Lagdo & Poechos

input_folder = Path("../data/raw/Water/Lagdo & Poechos/Rasters/Poechos_scene")
output_folder = "../data/processed/Water/Lagdo & Poechos/APNGs/test"

Z_MAX = 14
Z_MIN = 9
V_MAX = 1288
V_MIN = 0

palette = LinearSegmentedColormap.from_list(
    "palette",
    [
        "#2b83ba",
        "#abdda4",
        "#ffffbf",
        "#fdae61",
        "#d7191c"
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

animated_tiles = AnimatedTiles(
    data=str(input_folder),
    output_folder=str(output_folder),
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

#### Pakistan - Water

In [ ]:
# Pakistan
output_folder = "../data/processed/Water/Pakistan/APNGs/turbidity/"
input_folder = "../data/raw/Water/Pakistan/Rasters/turbidity"

Z_MAX = 13
Z_MIN = 12
V_MAX = 80
V_MIN = 0


colors = ["#018571",
        "#80cdc1",
        "#dfc27d",
        "#a6611a"
    ]
intervals = [0, 20, 40, 60, 80]

colors_rgba = np.array([hex_to_rgba(c) for c in colors], dtype=np.uint8)

# Normalize the bins to 0-255 scale
intervals_norm = np.clip((np.array(intervals) - V_MIN) / (V_MAX - V_MIN) * 255, 0, 255)

# Map every 0-255 value to its corresponding bin
cmap_uint8 = np.zeros((256, 4), dtype=np.uint8)
bins = intervals_norm
for i in range(256):
    # np.digitize returns the index of the bin the value belongs to
    idx = np.digitize(i, bins, right=False) - 1
    idx = max(0, min(idx, len(colors_rgba)-1))
    cmap_uint8[i] = colors_rgba[idx]

# Final colormap dictionary for rio-tiler
cm = {i: tuple(cmap_uint8[i]) for i in range(256)}

animated_tiles = AnimatedTiles(
    data=input_folder,
    output_folder=output_folder,
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

In [ ]:
# Pakistan-Balochistan: drought

input_folder = Path("../data/raw/Water/Pakistan/Rasters/drought/")
vector_file = Path("../data/raw/Water/Pakistan/Vectors/Dhadar_boundary.geojson")

# clip rasters by vector
clipped_folder = Path("../data/raw/Water/Pakistan/Rasters/drought_clipped/")
clip_rasters_by_vector(input_folder, vector_file, clipped_folder)

In [ ]:
# Pakistan-Balochistan: drought
input_folder = Path("../data/raw/Water/Pakistan/Rasters/drought_clipped/")
output_folder = "../data/processed/Water/Pakistan/APNGs/drought2/"

Z_MAX = 11
Z_MIN = 6
V_MAX = 2
V_MIN = -2

palette = LinearSegmentedColormap.from_list(
    "palette",
   [
    "#d7191c",
    "#fedf9a",
    "#ffffc0",
    "#aaaaaa",
    "#edf7af",
    "#dbf09e",
    "#1a9641"
],
    256,
)

x = np.linspace(0, 1, 255)  # only 255 indices because 0 = transparent
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")

# Build colormap dict: 0 → transparent
cm = {0: (0, 0, 0, 0)}
for idx, value in enumerate(cmap_uint8, start=1):
    cm[idx] = tuple(value)

animated_tiles = AnimatedTiles(
    data=str(input_folder),
    output_folder=str(output_folder),
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

#### Mexico - Water

In [ ]:
# Remove all positive values and NoData from Mexico NMDI rasters
input_folder = Path("../data/raw/Water/Mexico/Rasters/Drought/NMDI_zscores_seasonal_stats/")
output_folder = Path("../data/raw/Water/Mexico/Rasters/Drought/NMDI_negative/")
output_folder.mkdir(parents=True, exist_ok=True)

for tif in input_folder.glob("*.tif"):
    with rasterio.open(tif) as src:
        data = src.read(1)
        profile = src.profile

        # Set to transparent index (0):
        # - NoData value (127)
        # - All values > 0
        data = np.where((data == 127) | (data > 0), 0, data)

        profile.update(nodata=0)

        out_tif = output_folder / tif.name
        with rasterio.open(out_tif, "w", **profile) as dst:
            dst.write(data, 1)


In [ ]:
# Mexico
output_folder = "../data/processed/Water/Mexico/APNGs/nmdi/"
input_folder = "../data/raw/Water/Mexico/Rasters/Drought/NMDI_negative/"

Z_MAX = 10
Z_MIN = 7
V_MAX = 0
V_MIN = -2


palette = LinearSegmentedColormap.from_list(
    "palette",
    ["#d7191c", "#fdae61"],
    256
)

x = np.linspace(0, 1, 255)  # only 255 indices because 0 = transparent
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")

# Build colormap dict: 0 → transparent
cm = {0: (0, 0, 0, 0)}
for idx, value in enumerate(cmap_uint8, start=1):
    cm[idx] = tuple(value)

animated_tiles = AnimatedTiles(
    data=str(input_folder),
    output_folder=str(output_folder),
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

#### Sri Lanka - Water

In [ ]:
# Sri Lanka
output_folder = "../data/processed/Water/SriLanka/APNGs/min_extent/"
input_folder = "../data/raw/Water/SriLanka/Rasters/min_extent/"

Z_MAX = 13
Z_MIN = 12
V_MAX = 1
V_MIN = 0

# Simple binary colormap: 
cm = {}
for i in range(256):
    if i == 0:
        cm[i] = (0, 0, 0, 0)  # Transparent for value 0
    else:
        cm[i] = (0, 0, 255, 255)  # #0000FF blue for value 1

animated_tiles = AnimatedTiles(
    data=input_folder,
    output_folder=output_folder,
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

In [ ]:
# Sri Lanka
output_folder = "../data/processed/Water/SriLanka/APNGs/max_extent/"
input_folder = "../data/raw/Water/SriLanka/Rasters/max_extent/"

Z_MAX = 13
Z_MIN = 12
V_MAX = 1
V_MIN = 0

# Simple binary colormap: 
cm = {}
for i in range(256):
    if i == 0:
        cm[i] = (0, 0, 0, 0)  # Transparent for value 0
    else:
        cm[i] = (56, 173, 255, 255)  # #38ADFF blue for value 1

animated_tiles = AnimatedTiles(
    data=input_folder,
    output_folder=output_folder,
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

### Define variables: New

This section is to create a new layer. Once the layer is published, the subsection for that specific story can be moved to done.

#### New Story

In [ ]:
# New layer parameters


# Create animated-tiles from GeoTIFFs
animated_tiles = AnimatedTiles(
    data=input_folder,
    output_folder=output_folder,
    min_z=Z_MIN,
    max_z=Z_MAX,
    color_map=cm,
    vmin=V_MIN,
    vmax=V_MAX,
    engine="rasterio"
)

animated_tiles.create()

## Bucket management

Animated tiles need to be uploaded to the development bucket, within APNGs/. This section contains the code to manage the bucket's content.


### Upload files

In [ ]:
load_dotenv("/Users/sofia/Documents/Repos/esa/data-processing/.env")

development = "AWS1"
production = "AWS2"

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv(f"{development}_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv(f"{development}_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv(f"{development}_DEFAULT_REGION")
os.environ["AWS_BUCKET_NAME"] = os.getenv(f"{development}_BUCKET_NAME")


# Upload from local to S3
local_folder = "../data/processed/Water/SriLanka/APNGs/min_extent/"
s3_folder = "s3://esa-dev-public/APNGs/SriLankaMinExtent/"

subprocess.run(["aws", "s3", "cp", local_folder, s3_folder, "--recursive"])

### Delete files

In [ ]:
load_dotenv("/Users/sofia/Documents/Repos/esa/data-processing/.env")

development = "AWS1"
production = "AWS2"

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv(f"{development}_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv(f"{development}_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv(f"{development}_DEFAULT_REGION")
os.environ["AWS_BUCKET_NAME"] = os.getenv(f"{development}_BUCKET_NAME")

# Delete S3 folder
s3_folder = "s3://esa-dev-public/APNGs/MexicoNMDI/"

subprocess.run(["aws", "s3", "rm", s3_folder, "--recursive"])

### Transfer from one bucket to another

In [ ]:
load_dotenv("/Users/sofia/Documents/Repos/esa/data-processing/.env")

# Download data from one bucket to local
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("SECRET_KEY")

source_bucket = os.getenv("BUCKET_NAME")
source_endpoint = os.getenv("ENDPOINT_URL")

local_temp_folder = "../data/s3_transfer/"

subprocess.run([
    "aws", "s3", "cp",
    f"s3://{source_bucket}/",
    local_temp_folder,
    "--recursive",
    "--endpoint-url", source_endpoint,
], check=True)

In [ ]:
# Compare local folders with S3 folders to find which ones need to be uploaded
local_apngs = Path("../data/s3_transfer/APNGs")

local_folders = {
    p.name
    for p in local_apngs.iterdir()
    if p.is_dir()
}

bucket = "esa-dev-public"
prefix = "APNGs/"

result = subprocess.run(
    ["aws", "s3", "ls", f"s3://{bucket}/{prefix}"],
    capture_output=True,
    text=True,
    check=True,
)

bucket_folders = {
    line.strip()[4:].rstrip("/")  # Remove first 4 chars ("PRE ") and trailing "/"
    for line in result.stdout.splitlines()
    if line.strip().startswith("PRE ")
}


folders_to_upload = local_folders - bucket_folders

print("Folders to upload:")
print(folders_to_upload)

In [ ]:
# Upload missing folders to the new S3 bucket
for folder in folders_to_upload:
    local_path = f"../data/s3_transfer/APNGs/{folder}/"
    s3_path = f"s3://esa-dev-public/APNGs/{folder}/"

    print(f"Uploading {folder}...")
    subprocess.run(
        ["aws", "s3", "cp", local_path, s3_path, "--recursive"],
        check=True,
    )

In [ ]:
# Verify upload by listing folders again
local_apngs = Path("../data/s3_transfer/APNGs")

local_folders = {
    p.name
    for p in local_apngs.iterdir()
    if p.is_dir()
}

bucket = "esa-dev-public"
prefix = "APNGs/"

result = subprocess.run(
    ["aws", "s3", "ls", f"s3://{bucket}/{prefix}"],
    capture_output=True,
    text=True,
    check=True,
)

bucket_folders = {
    line.strip()[4:].rstrip("/")
    for line in result.stdout.splitlines()
    if line.strip().startswith("PRE ")
}

folders_to_upload = local_folders - bucket_folders

print("Folders to upload:")
print(folders_to_upload)

In [ ]:
# Get data that is not within APNGs in Digital Ocean
load_dotenv("/Users/sofia/Documents/Repos/esa/data-processing/.env")

# Download data from one bucket to local
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("SECRET_KEY")

source_bucket = os.getenv("BUCKET_NAME_CMS") # different bucket for non-APNGs
source_endpoint = os.getenv("ENDPOINT_URL")

local_temp_folder = "../data/s3_transfer/"

subprocess.run([
    "aws", "s3", "cp",
    f"s3://{source_bucket}/",
    local_temp_folder,
    "--recursive",
    "--endpoint-url", source_endpoint,
], check=True)

### Bucket management with boto3 - Deprecated

This section uses boto3 library to upload, delete or transfer data to the buckets. However, keep in mind that this library does not always work properly and it has caused issues when trying to access the bucket of the develop environment.

#### Delete files


In [ ]:
# destination_blob_path = "APNGs/SioniReservoir"
# delete_s3_folder(destination_blob_path, environment='staging')

#### Upload files

In [ ]:
# folder_path = "../data/processed/Water/Georgia/APNGs/SioniReservoir/"
# destination_blob_path = "APNGs/SioniReservoir2"
# upload_local_directory_to_s3(folder_path, destination_blob_path, environment='staging')

#### Migrate from one bucket to other

In [ ]:
# folder_path = "APNGs/PercentImperviousSurfaces"
# migrate_s3_folder(folder_path, dry_run=False)

#### Listing files in buckets

In [ ]:
# # Configure client in Digital Ocean bucket (staging)
# session = boto3.session.Session()
# client = session.client(
#     "s3",
#     config=botocore.config.Config(s3={"addressing_style": "virtual"}),
#     endpoint_url=os.getenv("ENDPOINT_URL"),
#     aws_access_key_id=os.getenv("KEY_ID"),
#     aws_secret_access_key=os.getenv("SECRET_KEY"),
# )

In [ ]:
# # Configure client in AWS bucket (production)
# session = boto3.session.Session()
# client = session.client(
#     "s3",
#     config=botocore.config.Config(s3={"addressing_style": "virtual"}),
#     endpoint_url=os.getenv("AWS_ENDPOINT_URL"),
#     aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
#     aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
#     region_name=os.getenv("AWS_DEFAULT_REGION"),
# )

In [ ]:
# # List All Buckets in a Region
# response = client.list_buckets()
# for space in response["Buckets"]:
#     print(space["Name"])

In [ ]:
# # List All Files in a Bucket
# response = client.list_objects(Bucket=os.getenv("BUCKET_NAME"))
# for obj in response["Contents"]:
#     print(obj["Key"])

In [ ]:
# # Configure S3 client based on environment
# def _get_s3_client(environment="staging"):
#     """Get configured S3 client."""
#     session = boto3.session.Session()
#     if environment == "staging":
#         return session.client(
#             "s3",
#             endpoint_url=os.getenv("ENDPOINT_URL"),
#             aws_access_key_id=os.getenv("KEY_ID"),
#             aws_secret_access_key=os.getenv("SECRET_KEY"),
#         )
#     elif environment == "production":
#         return session.client(
#             "s3",
#             endpoint_url=os.getenv("NEW_AWS_ENDPOINT_URL"),
#             aws_access_key_id=os.getenv("NEW_AWS_ACCESS_KEY_ID"),
#             aws_secret_access_key=os.getenv("NEW_AWS_SECRET_ACCESS_KEY"),
#             region_name=os.getenv("NEW_AWS_DEFAULT_REGION"),
#         )
#     else:
#         raise ValueError("Invalid environment specified for S3 client.")

# session = boto3.session.Session()
# client = _get_s3_client("staging")

# # List All Buckets in a Region
# response = client.list_buckets()
# for space in response["Buckets"]:
#     print(space["Name"])

# # List All Files in a Bucket
# response = client.list_objects(Bucket=os.getenv("NEW_AWS_BUCKET_NAME"))
# for obj in response["Contents"]:
#     print(obj["Key"])